In [72]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [126]:
import pandas as pd
import requests
import os
import sys
import datetime
import numpy as np
import re
import logging
from dateutil.relativedelta import relativedelta, TH
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
ohlcFl = r"/Users/shubhangi/DeepDocuments/Trading/NSEData/ohlcWithDelOI.csv"
yhfinanceFl = r"/Users/shubhangi/DeepDocuments/Trading/NSEData/correctedEODDataYF.csv"

In [74]:
ohlc = pd.read_csv(ohlcFl)
ohlc["date"] = pd.to_datetime(ohlc.date).apply(lambda x: x.date())
yFin = pd.read_csv(yhfinanceFl)
yFin["Date"] = pd.to_datetime(yFin.Date, format="%Y%m%d").apply(lambda x: x.date())

In [75]:
yFin.rename(
    columns={
        "Date": "date",
        "Ticker": "symbol",
        "Close": "corrClose",
        "High": "corrHigh",
        "Low": "corrLow",
        "Open": "corrOpen",
        "Volume": "yfVolume",
    },
    inplace=True,
)

In [76]:
finalOHLC = pd.merge(
    left=yFin,
    right=ohlc,
    left_on=["symbol", "date"],
    right_on=["symbol", "date"],
    how="outer",
)

In [77]:
def getNifty500():

    nList = "nifty500"
    url = "https://archives.nseindia.com/content/indices/ind_" + nList + "list.csv"

    return pd.read_csv(url)


nifty500 = getNifty500()

In [83]:
tenYearAgo = datetime.date.today() - relativedelta(years=10)
finalOHLC10yr = finalOHLC.loc[finalOHLC.date >= tenYearAgo, :]

In [84]:
finalOHLC10yrNifty500 = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(nifty500.Symbol), :
].dropna(subset=["corrOpen", "corrClose", "corrHigh", "corrLow"])

In [90]:
finalOHLC10yrNifty500 = finalOHLC10yrNifty500[
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ]
]
finalOHLC10yrNifty500.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
    },
    inplace=True,
)
finalOHLC10yrNifty500.head()

/Users/shubhangi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,date,symbol,close,high,low,open,volume,delivery,OI,company,industry
857156,2012-03-06,3MINDIA,4039.600098,4095.000000,3850.000000,3850.000000,815.0,608.0,NaN,3M India Ltd.,CONSUMER GOODS
857157,2012-03-06,AARTIDRUGS,9.784437,9.890790,9.731261,9.837613,701.0,580.0,NaN,Aarti Drugs Ltd.,PHARMA
857158,2012-03-06,AARTIIND,11.880698,11.942630,11.674257,11.674257,40837.0,20551.0,NaN,Aarti Industries Ltd.,CHEMICALS
857159,2012-03-06,ABB,760.551758,766.476658,694.954522,694.954522,79378.0,25918.0,356250.0,ABB India Ltd.,INDUSTRIAL MANUFACTURING
857160,2012-03-06,ABBOTINDIA,1439.524170,1469.277753,1364.087370,1364.087370,8043.0,3237.0,NaN,Abbott India Ltd.,PHARMA


In [149]:
finalOHLC10yrNifty500.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [94]:
finalOHLC10yrIndex = finalOHLC10yr.loc[
    finalOHLC10yr.industry == "INDEX",
    [
        "date",
        "symbol",
        "open",
        "high",
        "low",
        "close",
        "volume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrIndex.head()

,date,symbol,open,high,low,close,volume,delivery,OI,company,industry
1900626,2012-03-06,NSENIFTY,5266.00,5382.05,5206.40,5222.40,290396014.0,0.0,31265700.0,IND_NSENIFTY,INDEX
1900627,2012-03-07,NSENIFTY,5207.05,5243.85,5171.45,5220.45,230726017.0,0.0,30847300.0,IND_NSENIFTY,INDEX
1900628,2012-03-09,NSENIFTY,5294.10,5342.30,5291.60,5333.55,203005845.0,0.0,31297450.0,IND_NSENIFTY,INDEX
1900629,2012-03-12,NSENIFTY,5420.10,5421.90,5327.30,5359.55,176668237.0,0.0,31408000.0,IND_NSENIFTY,INDEX
1900630,2012-03-13,NSENIFTY,5391.05,5438.65,5390.80,5429.50,179965296.0,0.0,NaN,IND_NSENIFTY,INDEX


In [151]:
finalOHLC10yrIndex.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [101]:
othersImp = [
    "CRUDE_OIL",
    "GOLD",
    "HDFC_US_ADR",
    "ICICI_US_ADR",
    "INFOSYS_US_ADR",
    "USDINR",
    "DR_REDDY_US_ADR",
    "SILVER",
    "TATAMOTORS_US_ADR",
    "WIPRO_US_ADR",
]
finalOHLC10yrOther = finalOHLC10yr.loc[
    finalOHLC10yr.symbol.isin(othersImp),
    [
        "date",
        "symbol",
        "corrClose",
        "corrHigh",
        "corrLow",
        "corrOpen",
        "yfVolume",
        "delivery",
        "OI",
        "company",
        "industry",
    ],
]
finalOHLC10yrOther["company"] = "OTHERS"
finalOHLC10yrOther["industry"] = "OTHERS"
finalOHLC10yrOther.head()

,date,symbol,corrClose,corrHigh,corrLow,corrOpen,yfVolume,delivery,OI,company,industry
29389,2012-03-06,CRUDE_OIL,104.699997,107.339996,104.510002,107.110001,332561.0,NaN,NaN,OTHERS,OTHERS
29390,2012-03-06,GOLD,1671.400024,1708.699951,1666.000000,1708.699951,176.0,NaN,NaN,OTHERS,OTHERS
29391,2012-03-06,HDFC_US_ADR,15.369061,15.503959,15.252769,15.490003,2004200.0,NaN,NaN,OTHERS,OTHERS
29392,2012-03-06,ICICI_US_ADR,5.392008,5.457472,5.331334,5.417556,18514100.0,NaN,NaN,OTHERS,OTHERS
29393,2012-03-06,INFOSYS_US_ADR,5.077055,5.147165,5.052472,5.133508,9913600.0,NaN,NaN,OTHERS,OTHERS


In [152]:
finalOHLC10yrOther.drop_duplicates(subset=["symbol", "date"], inplace=True)

In [173]:
finalOHLC10yrOther.rename(
    columns={
        "corrClose": "close",
        "corrHigh": "high",
        "corrLow": "low",
        "corrOpen": "open",
        "yfVolume": "volume",
    },
    inplace=True,
)

In [170]:
corrDf = pd.DataFrame(columns=["stock", "testStock", "corr"])
for sym in finalOHLC10yrNifty500.symbol.unique():
    industry = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, "industry"
    ].values[0]
    stock = finalOHLC10yrNifty500.loc[
        finalOHLC10yrNifty500.symbol == sym, ["date", "symbol", "close"]
    ].sort_values(by=["date"])
    stock["ret"] = stock["close"].pct_change()
    stockNumOfDates = stock.date.shape[0]
    for testSym in finalOHLC10yrNifty500.loc[
        (
            (finalOHLC10yrNifty500.industry == industry)
            & (finalOHLC10yrNifty500.symbol != sym)
        ),
        :,
    ]["symbol"].unique():
        testStock = finalOHLC10yrNifty500.loc[
            finalOHLC10yrNifty500.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        noOfDates = testStock.shape[0]
        if stockNumOfDates == noOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )

    for testSym in finalOHLC10yrIndex.symbol.unique():
        testStock = finalOHLC10yrIndex.loc[
            finalOHLC10yrIndex.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
    for testSym in finalOHLC10yrOther.symbol.unique():
        testStock = finalOHLC10yrOther.loc[
            finalOHLC10yrOther.symbol == testSym, ["date", "symbol", "close"]
        ].sort_values(by=["date"])
        testNoOfDates = testStock.shape[0]
        if stockNumOfDates == testNoOfDates:
            testStock["ret"] = testStock["close"].pct_change()
            corr, _ = pearsonr(
                stock["ret"].dropna().values, testStock["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )
        else:
            commonDates = np.intersect1d(testStock.date, stock.date)

            stock1 = stock.loc[
                stock.date.isin(commonDates), ["date", "symbol", "close"]
            ]
            testStock1 = testStock.loc[
                testStock.date.isin(commonDates), ["date", "symbol", "close"]
            ]

            stock1["ret"] = stock1["close"].pct_change()
            testStock1["ret"] = testStock1["close"].pct_change()
            corr, _ = pearsonr(
                stock1["ret"].dropna().values, testStock1["ret"].dropna().values
            )
            corrDf = corrDf.append(
                pd.DataFrame({"stock": [sym], "testStock": [testSym], "corr": [corr]})
            )

    # break

In [179]:
corrDf.sort_values(by=["corr"], ascending=False).to_csv(
    r"/Users/shubhangi/DeepDocuments/Trading/NSEData/corr.csv", index=False
)

In [180]:
finalOHLC10yrOther.append(finalOHLC10yrIndex).append(finalOHLC10yrNifty500).to_csv(
    r"/Users/shubhangi/DeepDocuments/Trading/NSEData/nifty500OtherOHLC10yrs.csv",
    index=False,
)